# Part 1
## Decision Trees (aka Classification and Regression Trees)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

In [ ]:


# Read in the data.
path = './data/titanic.csv'
titanic = pd.read_csv(path)

# Encode female as 0 and male as 1.
titanic['Sex'] = titanic.Sex.map({'female':0, 'male':1})

# Fill in the missing values for age with the median age.
titanic.Age.fillna(titanic.Age.median(), inplace=True)

# Create a DataFrame of dummy variables for Embarked.
titanic = pd.get_dummies(titanic, columns=['Embarked'])


# create predictor matrix and target
feature_cols = ['Pclass', 'Sex', 'Age', 'Embarked_Q', 'Embarked_S','Embarked_C']
X = titanic[feature_cols]
y = titanic.Survived

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# instantiate - random_state for reproducibility, max_depth as a stopping criterion
treeclf = DecisionTreeClassifier(max_depth=3, random_state=1)
# fit
treeclf.fit(X, y)

In [ ]:
titanic.head(1)

## How Does a Computer Build a Tree?

1. Begin at the top of the tree.
2. For **every feature**, examine **every possible cutpoint**, and choose the feature and cutpoint so that the resulting tree has the greatest decrease in impurity. Make that split.
3. Examine the two resulting regions. Once again, make a **single split** (in one of the regions) to minimize the MSE.
4. Keep repeating Step 3 until a **stopping criterion** is met:
    - Maximum tree depth (maximum number of splits required to arrive at a leaf).
    - Minimum number of observations in a leaf.
    - All the points are classified

---
**Ideal approach:** Considering every possible partition of the feature space (computationally infeasible).

**"Good enough" approach:** Take the best split at each step (recursive binary splitting.)

This is a **greedy algorithm** because it makes locally optimal decisions -- it takes the best split at each step. A greedy algorithm hopes that a series of locally optimal decisions might be optimal overall; however, this is not always the case. 

![Tree for Titanic data](assets/tree_titanic.png)

- The **maximum value** of the Gini index is 0.5 and occurs when the classes are perfectly balanced in a node.
- The **minimum value** of the Gini index is 0 and occurs when there is only one class represented in a node.
- A node with a lower Gini index is said to be more "pure.

- how would a decision tree **regressor** assess how well separated the data is?

### Evaluating our model
####  The importance of a feature is computed as the (normalized) total reduction of the criterion brought by that feature. In other words, the higher the feature importance, the more effective the feature was able to split the data to create more pure splits

In [ ]:
dtc_fi = pd.DataFrame({'feature':feature_cols, 'importance':treeclf.feature_importances_})
dtc_fi.sort_values('importance',ascending=False)

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

# decision trees have lots of hyperparameters that we could potentially optimize
# using cross validation to optimize hyperparameters (tune our model)
for depth in [2,6,None]:
    treeclf = DecisionTreeClassifier(max_depth=depth)
    scores = cross_val_score(treeclf, X, y, cv=5)
    print('at max_depth of '+str(depth)+': '+str(np.mean(scores))+ ' +/- ' +str(np.std(scores)))

### If we don't establish a stopping criterion, what is a decision tree going to do? 

### Add decision trees to machine learning framework

# Random Forest

https://www.youtube.com/watch?v=loNcrMjYh64

### What is Ensembling?

**Ensemble learning (or "ensembling")** is the process of combining several predictive models in order to produce a combined model that is more accurate than any individual model. For example, given predictions from several models we could:

- **Regression:** Take the average of the predictions.
- **Classification:** Take a vote and use the most common prediction.

## Bagging

A **bootstrap sample** is a random sample with replacement. Think flipping a coin as opposed to hitting shuffle on a playlist. So, it has the same size as the original sample but might duplicate some of the original observations.

**Bagging** is short for **bootstrap aggregation**, meaning the aggregation of bootstrap samples.


In [ ]:
# Set a seed for reproducibility.
np.random.seed(1)

# Create an array of 1 through 20.
nums = np.arange(1, 21)
print(nums)

# Sample that array 20 times with replacement.
print(np.random.choice(a=nums, size=20, replace=True))

# For each tree, the **unused observations** are called "out-of-bag" observations.

**How does bagging work (for decision trees)?**



**Random Forest

1. Grow B trees using B bootstrap samples from the training data.
2. Train each tree on its bootstrap sample 
3. When building each tree, each time a split is considered, a **random sample of m features** is chosen as split candidates from the **full set of p features**. The split is only allowed to use **one of those m features**. A new random sample of features is chosen for **every single tree at every single split**.
3. Combine the predictions:
    - Average the predictions for **regression trees**.
    - Take a vote for **classification trees**.


**What's the point?**

* If every batch of test data was exactly like our training data, what would be better, CART or Random Forest?
* Why bother with Random Forest?

In [ ]:
rfc_best = RandomForestClassifier(max_depth=4, max_features=4, min_samples_split=5)
rfc_best.fit(X, y)
rfc_best.score(X,y)

In [ ]:
pd.DataFrame({'feature':feature_cols, 'importance':rfc_best.feature_importances_}).sort_values('importance',ascending=False)

### Add decision trees to machine learning framework